# Packages 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import cv2
import dlib
import math
from math import cos, sin
import joblib

# Load Data

In [2]:
df_pose=pd.read_csv('data/head_pose.csv')
df_pose.head()

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,yaw,pitch,roll
0,121.868034,122.367607,126.819237,130.831787,137.523132,148.521729,161.528015,182.876678,213.940063,248.005371,...,310.149048,313.233856,315.107117,322.384613,330.965576,330.701965,324.903870,1.044306,-22.874239,4.908886
1,281.238159,277.339417,274.876953,269.523773,257.788269,240.968155,221.801483,205.846298,207.085449,226.185638,...,303.093811,300.724457,300.060974,303.260895,299.485168,300.598602,302.286499,68.155240,26.932743,17.243670
2,236.385101,239.198257,242.566376,243.971375,242.828186,238.567902,232.561859,229.155289,238.303162,259.122467,...,303.122009,302.877289,302.146454,300.489380,303.481873,304.681732,304.631042,50.485413,-10.579652,-13.570645
3,168.029221,177.697510,190.641602,201.395294,211.957214,222.723099,231.350220,244.332855,268.327637,291.832214,...,315.469391,312.701294,310.133301,300.041290,315.544800,319.498596,320.843994,17.143373,-10.048455,-21.392782
4,280.462250,287.249817,293.892456,297.539368,295.677002,287.270355,271.661591,254.701157,250.219299,261.212463,...,306.811768,307.014893,305.351318,309.934814,311.558899,312.920746,312.058350,68.640550,-50.544582,-59.207973


# Feature & targets

In [3]:
y=df_pose[['yaw','pitch','roll']]
x=df_pose.drop(columns=['yaw','pitch','roll'],axis=1)

# Data preprocessing

In [4]:
x_coordinate_idx=[str(i) for i in range(x.shape[1]) if i%2==0]
y_coordinate_idx=[str(i) for i in range(x.shape[1]) if i%2!=0]

In [5]:
class SubImputer(SimpleImputer):
    def __init__(self,x_num,x_corr,y_num,y_corr):
        self.x_num= x_num
        self.y_num= y_num
        self.x_corr=x_corr
        self.y_corr=y_corr
        return 

    def fit(self, X):
        
        return self

    def transform(self, X):
        #seperate x&y points
        x_data=X[self.x_corr]
        y_data=X[self.y_corr]
        #subtract the centre point
        x_data=x_data.sub(x_data[f'{self.x_num}'],axis=0)
        y_data=y_data.sub(y_data[f'{self.y_num}'],axis=0)
        #join the two data frames
        full_data=pd.concat([x_data, y_data], axis=1)
        #resort the coloumns
        data_idx=[str(i) for i in range(X.shape[1])]
        
        return full_data[data_idx].sort_index(axis = 0)
    
    def predict(self,X):
        return self.transform(X)

In [6]:
class NormaliseImputer(SimpleImputer):
    def __init__(self,x_corr,y_corr):
        self.x_corr=x_corr
        self.y_corr=y_corr
        return 

    def fit(self, X):
        
        return self

    def transform(self, X):
        #seperate x&y points
        x_data=X[self.x_corr]
        y_data=X[self.y_corr]
        #normalise the points => p-min/max-min
        x_data=x_data.sub(x_data.min(axis=1),axis=0)
        x_data=x_data.div( (x_data.max(axis=1)-x_data.min(axis=1)) ,axis=0)
        
        y_data=y_data.sub(y_data.min(axis=1),axis=0)
        y_data=y_data.div( (y_data.max(axis=1)-y_data.min(axis=1)) ,axis=0)
        #join the two data frames
        full_data=pd.concat([x_data, y_data], axis=1)
        #resort the coloumns
        data_idx=[str(i) for i in range(X.shape[1])]
        
        return full_data[data_idx].sort_index(axis = 0)
    
    def predict(self,X):
        return self.transform(X)

In [7]:
preprocessing_pip =Pipeline([
                              ("noramalise the face points",NormaliseImputer(x_coordinate_idx,y_coordinate_idx)),
                              ("centre the coordinate arround the nose",SubImputer(30,x_coordinate_idx,31,y_coordinate_idx)),
                            ])

# Build the model

In [8]:
para_grid=[
            {'bootstrap':[True]},
            {'max_leaf_nodes':[i for i in range(30,100)] },
            {'n_estimators':[i for i in range(40,100)]},
            {'max_samples':[i for i in range(90,1000)]},
            { 'n_jobs':[-1]}
]

para_grid_2=[
            {'bootstrap':[True]},
            {'max_leaf_nodes':[i for i in range(70,120)] },
            {'n_estimators':[i for i in range(100,250)]},
            {'max_samples':[i for i in range(800,1250)]},
            { 'n_jobs':[-1]}
]

In [9]:
x_p= preprocessing_pip.fit_transform(x)

In [10]:
#choose best parameters using grid search

In [11]:
rand_search_yaw=RandomizedSearchCV(RandomForestRegressor(),para_grid,cv=5,scoring="neg_mean_squared_error")
rand_search_pitch=RandomizedSearchCV(RandomForestRegressor(),para_grid_2,cv=5,scoring="neg_mean_squared_error")
rand_search_roll=RandomizedSearchCV(RandomForestRegressor(),para_grid,cv=5,scoring="neg_mean_squared_error")

In [12]:
_=rand_search_yaw.fit(x_p,y.yaw)
_=rand_search_roll.fit(x_p,y.roll)
_=rand_search_pitch.fit(x_p,y.pitch)

In [13]:
#test the choosen models

In [11]:
yaw_model=rand_search_yaw.estimator
_=yaw_model.fit(x_p,y.yaw)
score = yaw_model.score(x_p, y.yaw)
print("R-squared:", score) 

NameError: name 'rand_search_yaw' is not defined

In [12]:
roll_model=rand_search_roll.estimator
_=roll_model.fit(x_p,y.roll)
score = roll_model.score(x_p, y.roll)
print("R-squared:", score) 

R-squared: 0.9491116258687262


In [16]:
pitch_model=rand_search_pitch.estimator
_=pitch_model.fit(x_p,y.pitch)
score = pitch_model.score(x_p, y.pitch)
print("R-squared:", score) 

R-squared: 0.8331659985693691


In [17]:
#save the modles

In [20]:
_=joblib.dump(yaw_model,"yaw_model.pkl")
_=joblib.dump(roll_model,"roll_model.pkl")
_=joblib.dump(pitch_model,"pitch_model.pkl")

In [21]:
#load the models

In [10]:
yaw_model =joblib.load("yaw_model.pkl")
roll_model =joblib.load("roll_model.pkl")
pitch_model =joblib.load("pitch_model.pkl")

# Full pipeline

In [13]:
yaw_pip=Pipeline([
                    ('preprocessing',preprocessing_pip),
                    ('yaw model',yaw_model)
                 ])

In [14]:
pitch_pip=Pipeline([
                    ('preprocessing',preprocessing_pip),
                    ('yaw model',pitch_model)
                 ])

In [15]:
roll_pip=Pipeline([
                    ('preprocessing',preprocessing_pip),
                    ('yaw model',roll_model)
                 ])

# Generate & draw landmarks

In [25]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 2500):
    
    pitch = pitch * np.pi / 180
    yaw = -(yaw * np.pi / 180)
    roll = roll * np.pi / 180

    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    
    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [60]:
def generate_landmarks(img): 
    img=cv2.resize(img,(512,512))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(img_gray)
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    faces = detector(img_gray)
    if len(faces)==0:
        return l 
    for face in faces:
        landmarks = predictor(img_gray, face)
        landmarks_points = []
        landmarks_points_x = []
        landmarks_points_y = []
        for n in range(0,68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            landmarks_points_x.append(x)
            landmarks_points_y.append(y)
            cv2.circle(img, (x,y), radius=0, color=(0,0,0), thickness=5)
        #cv2.imshow(path,img)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()   
    landmarks_points=landmarks_points_x+landmarks_points_y
    landmarks_points=np.array(landmarks_points).reshape((1,136))
    landmarks=pd.DataFrame(landmarks_points,columns=df_pose.columns[0:136])
    return landmarks

In [61]:
def draw_axis_cam():
    vid = cv2.VideoCapture(0)
    while(True):
        ret, frame = vid.read()
        #get landmarks
        landmarks=generate_landmarks(frame)
        #predict yaw,pitch and roll
        yaw_p=yaw_pip.predict(landmarks)
        pitch_p=pitch_pip.predict(landmarks)
        roll_p=roll_pip.predict(landmarks)
        #draw axis
        frame=draw_axis(frame, pitch_p,yaw_p,roll_p)
        #show the frame
        cv2.imshow('webCam', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            vid.release()
            cv2.destroyAllWindows()
            return
            
    
        

In [62]:
def draw_axis_video(path):
    cap = cv2.VideoCapture(path)
    # Check if camera opened successfully
    if (cap.isOpened()== False):
        print("Error opening video stream or file")

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            #get landmarks
            landmarks=generate_landmarks(frame)
            #predict yaw,pitch and roll
            yaw_p=yaw_pip.predict(landmarks)
            pitch_p=pitch_pip.predict(landmarks)
            roll_p=roll_pip.predict(landmarks)
            #draw axis
            frame=draw_axis(frame, pitch_p,yaw_p,roll_p)
            #show the frame
            cv2.imshow('Frame',frame)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                  break

        else:
            break

        
    cap.release()
    cv2.destroyAllWindows()
    return

In [59]:
l=generate_landmarks(cv2.imread('alkber.jpg'))

In [64]:
draw_axis_video('alkber360.mp4')

In [43]:
img=cv2.imread('alkber.jpg')

landmarks=generate_landmarks(img)

yaw_p=yaw_pip.predict(landmarks)
pitch_p=pitch_pip.predict(landmarks)
roll_p=roll_pip.predict(landmarks)
print(yaw_p,pitch_p,roll_p)
img=draw_axis(img,pitch_p,yaw_p,roll_p) 
cv2.imshow('test',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

ddd
[0.4345231] [-1.13715275] [13.726375]
